# Electricity Contract Selection

## INTRODUCTION
###### Just over a year ago you had a smart electricity meter installed at your home. This particular model of
###### smart meter reports your usage to your electricity provider every hour. Your electricity contract is
###### due for renewal and being a top financial analyst you decide to identify which contract will minimize
###### your annual electricity cost.
###### There are 3 different types of electricity contracts available to you:
###### • No Flex: The cost per kWh of electricity is constant for the entire year
###### • Monthly Flex: The cost per kWh of electricity fluctuates depending on the month
###### • Hourly Flex: The cost per kWh of electricity fluctuates based on the time of day
###### To decide which contract is optimal, you decide to compare the costs under each of the
###### contracts assuming your usage remains exactly the same as last year. You've contacted your
###### current electricity supplier to obtain your electricity usage history for the last year.
###### Unfortunately, they provided it to you in a poorly structured and unsorted format,
###### so you will need to clean it up prior to doing your analysis.
###### In the data the hour field identifies when the hour starts, so 8AM would be from 8:00AM
###### until 8:59AM.
##### SUPPORTING DATA
###### You’ve placed your usage history and the specifics of each of the electricity contracts into the
###### supplied workbook. Your usage history is located on the “Usage” sheet while the specifics about
###### each of the electricity contracts

# Importing libraries

In [6]:
import re
from datetime import datetime

import pandas as pd
import numpy as np

# Loading Data

In [7]:
contracts_df = pd.read_excel('data_cleaning.xlsx','Contracts',header=None)
usage_df = pd.read_excel('data_cleaning.xlsx','Usage',header=None)

In [8]:
usage_df

,0
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [9]:
usage_df.columns=['xx']
usage_df['xx']=usage_df['xx'].str.lower().str.strip()
usage_df[5000:5035]

,xx
5000,__5am fri 28th-nov-2014_0.48 kwh
5001,9am 11-jun-2014____0.567 kwh
5002,_1am monday 5-may-2014___0.141 kwh
5003,__12am sunday 9-nov-2014_0.299 kwh
5004,5pm 03-nov-2014_0.491 kwh
5005,_5 pm 19th-may-2014__1.102kwh
5006,9 am tue 08th-jul-2014___0.864 kwh
5007,3 pm thu 23-oct-2014___0.202 kwh
5008,_11pm wed 6th-aug-2014_0.534 kwh
5009,_4pm 03-may-2014___0.85 kwh


# Data Cleaning

In [10]:
usage_df.duplicated().sum()

6

In [11]:
usage_df.drop_duplicates(inplace=True)

In [12]:
# separating values using regex
pattern = re.compile(r"(?:[\s\-\_]*)(\d+)(?:[\s\-\_]*)(am|pm)(?:[a-z\s]*)(\d{1,2})(?:.*)([a-z]{3})(?:[\s\-\_]*)(\d{4})(?:[\s\-\_]*)(\d+\.?\d+)")

def parse(df, pattern):
    text = df['xx']
    matched = pattern.match(text)
    return int(matched.group(1)), str(matched.group(2)), int(matched.group(3)), str(matched.group(4)), int(matched.group(5)), float(matched.group(6))

usage=usage_df.apply(func = parse, args = (pattern,), axis=1, result_type = 'expand')
usage.columns = ['hour', 'am_pm', 'day', 'month', 'year', 'kwh']

In [13]:
usage

,hour,am_pm,day,month,year,kwh
0,3,pm,24,mar,2014,0.384
1,5,am,15,aug,2014,1.201
2,8,pm,20,mar,2014,1.523
3,6,pm,23,apr,2014,0.424
4,1,am,19,dec,2014,0.209
...,...,...,...,...,...,...
8755,1,am,7,nov,2014,0.084
8756,6,am,20,may,2014,1.027
8757,2,am,8,apr,2014,0.052
8758,9,pm,27,jan,2014,1.428


In [14]:
#creating datetime column
def to_datetime_(df):
    dt=f"{df['year']}-{df['month']}-{df['day']}-{df['hour']}-{df['am_pm']}"
    return pd.to_datetime(datetime.strptime(dt,"%Y-%b-%d-%I-%p"))

In [15]:
usage['datetime']=usage.apply(func=to_datetime_,axis=1,result_type='expand')

In [16]:
usage

,hour,am_pm,day,month,year,kwh,datetime
0,3,pm,24,mar,2014,0.384,2014-03-24 15:00:00
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00
2,8,pm,20,mar,2014,1.523,2014-03-20 20:00:00
3,6,pm,23,apr,2014,0.424,2014-04-23 18:00:00
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00
...,...,...,...,...,...,...,...
8755,1,am,7,nov,2014,0.084,2014-11-07 01:00:00
8756,6,am,20,may,2014,1.027,2014-05-20 06:00:00
8757,2,am,8,apr,2014,0.052,2014-04-08 02:00:00
8758,9,pm,27,jan,2014,1.428,2014-01-27 21:00:00


In [17]:
#creating column for day of the week
usage['d_o_w']=usage['datetime'].dt.dayofweek.map({0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'})
usage

,hour,am_pm,day,month,year,kwh,datetime,d_o_w
0,3,pm,24,mar,2014,0.384,2014-03-24 15:00:00,Monday
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,Friday
2,8,pm,20,mar,2014,1.523,2014-03-20 20:00:00,Thursday
3,6,pm,23,apr,2014,0.424,2014-04-23 18:00:00,Wednesday
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,Friday
...,...,...,...,...,...,...,...,...
8755,1,am,7,nov,2014,0.084,2014-11-07 01:00:00,Friday
8756,6,am,20,may,2014,1.027,2014-05-20 06:00:00,Tuesday
8757,2,am,8,apr,2014,0.052,2014-04-08 02:00:00,Tuesday
8758,9,pm,27,jan,2014,1.428,2014-01-27 21:00:00,Monday


In [18]:
#creating 24 hour time column
usage['hour']=usage['datetime'].dt.hour
usage

,hour,am_pm,day,month,year,kwh,datetime,d_o_w
0,15,pm,24,mar,2014,0.384,2014-03-24 15:00:00,Monday
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,Friday
2,20,pm,20,mar,2014,1.523,2014-03-20 20:00:00,Thursday
3,18,pm,23,apr,2014,0.424,2014-04-23 18:00:00,Wednesday
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,Friday
...,...,...,...,...,...,...,...,...
8755,1,am,7,nov,2014,0.084,2014-11-07 01:00:00,Friday
8756,6,am,20,may,2014,1.027,2014-05-20 06:00:00,Tuesday
8757,2,am,8,apr,2014,0.052,2014-04-08 02:00:00,Tuesday
8758,21,pm,27,jan,2014,1.428,2014-01-27 21:00:00,Monday


In [19]:
usage.sort_values(by='datetime',inplace=True)
usage.reset_index(drop=True,inplace=True)

In [20]:
usage

,hour,am_pm,day,month,year,kwh,datetime,d_o_w
0,0,am,1,jan,2014,0.684,2014-01-01 00:00:00,Wednesday
1,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,Wednesday
2,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,Wednesday
3,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,Wednesday
4,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,Wednesday
...,...,...,...,...,...,...,...,...
8749,19,pm,31,dec,2014,1.940,2014-12-31 19:00:00,Wednesday
8750,20,pm,31,dec,2014,1.901,2014-12-31 20:00:00,Wednesday
8751,21,pm,31,dec,2014,0.766,2014-12-31 21:00:00,Wednesday
8752,22,pm,31,dec,2014,0.633,2014-12-31 22:00:00,Wednesday


### Average hourly electricity usage

In [21]:
round(usage['kwh'].mean(),3)

0.782

### Average electricity usage per hour in February

In [22]:
usage[usage['month']=='feb']['kwh'].mean()

0.833734724292103

### Day of the week with the highest average usage

In [23]:

usage.groupby('d_o_w').mean().kwh.sort_values(ascending=False).head(1)

d_o_w
Sunday    0.898601
Name: kwh, dtype: float64

### Highest amount of electricity used in a continuous 4 hour period

In [24]:
value = []
for i in range(usage.shape[0]):
    value.append(usage['kwh'].iloc[i:i+4].sum()) 
                        
round(max(value),3)

17.237

In [25]:
usage['kwh'].rolling(4).sum().max()

17.237000000000002

### Based on the historic electricity usage, what would annual cost of electricity be under the "Monthly Flex" contract?
#### Monthly Flex: The cost per kWh of electricity fluctuates depending on the month

In [26]:
contracts=contracts_df[10:]
contracts
monthly=contracts.iloc[2:14,[4,5,6]]
monthly.reset_index(drop=True,inplace=True)
monthly

,4,5,6
0,January,0.20,$/kWh
1,February,0.19,$/kWh
2,March,0.17,$/kWh
3,April,0.18,$/kWh
4,May,0.22,$/kWh
5,June,0.27,$/kWh
6,July,0.24,$/kWh
7,August,0.19,$/kWh
8,September,0.18,$/kWh
9,October,0.15,$/kWh


In [27]:
usage.groupby('month').sum().reset_index().merge(monthly, left_on='month', right_on=4)

,month,hour,day,year,kwh,4,5,6


In [28]:
usage['month']=usage['datetime'].dt.month_name(locale = 'English')

In [29]:
annual=usage.groupby('month').sum().reset_index().merge(monthly, left_on='month', right_on=4)
annual

,month,hour,day,year,kwh,4,5,6
0,April,8280,11160,1450080,151.242,April,0.18,$/kWh
1,August,8556,11904,1498416,633.390,August,0.19,$/kWh
2,December,8556,11904,1498416,568.739,December,0.19,$/kWh
3,February,7725,9740,1351394,559.436,February,0.19,$/kWh
4,January,8556,11904,1498416,1042.660,January,0.20,$/kWh
5,July,8544,11878,1496402,1254.055,July,0.24,$/kWh
6,June,8249,11094,1444038,899.904,June,0.27,$/kWh
7,March,8572,11928,1500430,421.132,March,0.17,$/kWh
8,May,8533,11893,1496402,416.813,May,0.22,$/kWh
9,November,8280,11160,1450080,232.420,November,0.14,$/kWh


In [30]:
monthly_flex=sum(annual['kwh']*annual[5])
monthly_flex

1420.0582800000002

### Based on your historic electricity usage, which of the three contracts would pro- duce the lowest annual cost?
a.The No Flex plan
b. The Monthly Flex plan
c. The Hourly Flex plan
d. Impossible to Determine

In [31]:
hourly_usage=usage.groupby('hour').sum().reset_index()
hourly_usage

,hour,day,year,kwh
0,0,5738,735110,181.638
1,1,5719,733096,93.757
2,2,5738,735110,92.192
3,3,5734,733096,92.315
4,4,5738,735110,93.596
5,5,5738,735110,352.153
6,6,5738,735110,446.288
7,7,5738,735110,399.243
8,8,5738,735110,268.043
9,9,5738,735110,190.439


In [32]:
flat_rate=sum(annual['kwh']*0.21)
flat_rate,monthly_flex

(1436.8689299999999, 1420.0582800000002)

In [33]:
hourly=contracts.iloc[2:,8:].reset_index(drop=True)
hourly[8]=np.arange(0,24)
hourly

,8,9,10
0,0,0.10,$/kWh
1,1,0.10,$/kWh
2,2,0.10,$/kWh
3,3,0.10,$/kWh
4,4,0.15,$/kWh
5,5,0.20,$/kWh
6,6,0.24,$/kWh
7,7,0.24,$/kWh
8,8,0.26,$/kWh
9,9,0.20,$/kWh


In [34]:
hourly_flex=hourly_usage.merge(hourly, left_on='hour', right_on=8)

In [35]:
hourly_flex_price=sum(hourly_flex.kwh*hourly_flex[9])

In [36]:
#Comparing the annual costs of each flex packages
print('Annual cost of the Monthly Flex option: ', monthly_flex)
print('Annual cost of the Hourly Flex option: ', hourly_flex_price)
print('Annual cost of the Flat rate option: ', flat_rate)

Annual cost of the Monthly Flex option:  1420.0582800000002
Annual cost of the Hourly Flex option:  1367.84829
Annual cost of the Flat rate option:  1436.8689299999999


In [37]:
import session_info


session_info.show()